In [1]:
from __future__ import print_function

import glob
from os import path
from time import perf_counter as clock

from astropy.io import fits
from scipy import ndimage
import numpy as np
import matplotlib.pyplot as plt

import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util

import gc
gc.collect()



0

In [2]:
ppxf_dir   = "/home/carlos/Downloads/BADASS3-master/badass_data_files/indo_us_library/"
spec_path  = "/home/carlos/Desktop/pPXF(SDP)/Data/Voronoi20/Binned_Spec/"
save_paths = glob.glob("/home/carlos/Desktop/pPXF(SDP)/Data/Voronoi20/Bootstrapping/Bin*/")

In [4]:
def _fit(spec_name, galboots=None, bootstrapping=False):
    # Read a galaxy spectrum and define the wavelength range
    #
    file = spec_path + spec_name + ".fits"
    hdu = fits.open(file)
    gal_lin = hdu[1].data
    h1 = hdu[1].header

    lamRange1 = h1['CRVAL1'] + np.array([0., h1['CDELT1']*(h1['NAXIS1'] - 1)])
    FWHM_gal = 2.71  # SAURON has an instrumental resolution FWHM of 4.2A.

    #READING THE VARIANCE/LENDO A VARIÂNCIA DO ESPECTRO

    var = hdu[2].data     #Read the variance/Lendo a variância
    h_var = hdu[2].header #Read the variance header/Lendo o header da variância
    lamRange_var = h_var['CRVAL1'] + np.array([0.,h_var['CDELT1']*(h_var['NAXIS1']-1)])

    # If the galaxy is at significant redshift, one should bring the galaxy
    # spectrum roughly to the rest-frame wavelength, before calling pPXF
    # (See Sec2.4 of Cappellari 2017). In practice there is no
    # need to modify the spectrum in any way, given that a red shift
    # corresponds to a linear shift of the log-rebinned spectrum.
    # One just needs to compute the wavelength range in the rest-frame
    # and adjust the instrumental resolution of the galaxy observations.
    # This is done with the following three commented lines:
    #
    z = 0.299  # Initial redshift estimate of the galaxy
    #z = 1.23 # Initial estimate of the galaxy redshift
    lamRange1 = lamRange1/(1+z) # Compute approximate restframe wavelength range
    FWHM_gal = FWHM_gal/(1+z)   # Adjust resolution in Angstrom
    lamRange_var = lamRange_var/(1+z)
    z = 0.00

    galaxy, logLam1, velscale = util.log_rebin(lamRange1, gal_lin)
    if bootstrapping is True:
        galaxy = galboots
        
    median_galaxy = np.median(galaxy)
    galaxy = galaxy/median_galaxy  # Normalize spectrum to avoid numerical issues
    lam = np.exp(logLam1)


    erro = np.sqrt(var) #1-sigma error from variance/1-sigma erro da variância
    erro, loglam_var, velscale_var = util.log_rebin(lamRange_var, erro)
    noise = erro/median_galaxy         #Normalizing the error/Normalizando o erro    

    # Read the list of filenames from the Single Stellar Population library
    # by Vazdekis (2010, MNRAS, 404, 1639) http://miles.iac.es/. A subset
    # of the library is included for this example with permission
    vazdekis = glob.glob(ppxf_dir + '/*.fits')
    FWHM_tem = 1.35  # Indu spectra  resolution
    velscale_ratio = 2  # adopts 2x higher spectral sampling for templates than for galaxy
    
    
    # Extract the wavelength range and logarithmically rebin one spectrum
    # to a velocity scale 2x smaller than the SAURON galaxy spectrum, to determine
    # the size needed for the array which will contain the template spectra.
    #
    hdu = fits.open(vazdekis[0])
    ssp = hdu[0].data
    h2 = hdu[0].header
    lamRange2 = h2['CRVAL1'] + np.array([0., h2['CDELT1']*(h2['NAXIS1'] - 1)])
    sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
    templates = np.empty((sspNew.size, len(vazdekis)))
    
    # Convolve the whole Vazdekis library of spectral templates
    # with the quadratic difference between the SAURON and the
    # Vazdekis instrumental resolution. Logarithmically rebin
    # and store each template as a column in the array TEMPLATES.

    # Quadratic sigma difference in pixels Vazdekis --> SAURON
    # The formula below is rigorously valid if the shapes of the
    # instrumental spectral profiles are well approximated by Gaussians.
    #
    FWHM_dif = np.sqrt(FWHM_gal**2 - FWHM_tem**2)
    sigma = FWHM_dif/2.355/h2['CDELT1']  # Sigma difference in pixels

    for j, file in enumerate(vazdekis):
        hdu = fits.open(file)
        ssp = hdu[0].data
        ssp = ndimage.gaussian_filter1d(ssp, sigma)
        sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
        templates[:, j] = sspNew/np.median(sspNew)  # Normalizes templates
        
        # The galaxy and the template spectra do not have the same starting wavelength.
    # For this reason an extra velocity shift DV has to be applied to the template
    # to fit the galaxy spectrum. We remove this artificial shift by using the
    # keyword VSYST in the call to PPXF below, so that all velocities are
    # measured with respect to DV. This assume the redshift is negligible.
    # In the case of a high-redshift galaxy one should de-redshift its
    # wavelength to the rest frame before using the line below (see above).
    #
    c = 299792.458
    dv = (np.mean(logLam2[:velscale_ratio]) - logLam1[0])*c  # km/s

    goodPixels = util.determine_goodpixels(logLam1, lamRange2, z)
    # Here the actual fit starts. The best fit is plotted on the screen.
# Gas emission lines are excluded from the pPXF fit using the GOODPIXELS keyword.

    vel = c*np.log(1 + z)   # eq.(8) of Cappellari (2017)
    start = [vel, 200.]  # (km/s), starting guess for [V, sigma]


    pp = ppxf(templates, galaxy, noise, velscale, start,lam=lam,
              goodpixels=goodPixels, plot=True, moments=2,quiet=True,
              degree=6, vsyst=dv, velscale_ratio=velscale_ratio, bias=0.)
    #print("Formal errors:")
    #print("     dV    dsigma   dh3      dh4")
    #print("".join("%8.2g" % f for f in pp.error*np.sqrt(pp.chi2)))
    hdu.close()
    out = pp.bestfit, pp.galaxy, pp.sol, pp.noise, goodPixels
    return out

In [5]:
def bootstrap_fig(save_path, spec_name, it,
                                 galboots=None):
    fig = plt.figure()
    
    if galboots is not None:
        out = _fit(spec_name, galboots=galboots, bootstrapping=True)
    else:
        out = _fit(spec_name)
    sols = out[2]
    
    textx = 500
    texty = 1.2
    sols = out[2]
    textx = [375, 385]
    texty = 1.4
    texty2 = 1.3
    fs = 20
    plt.text(textx[0], texty, r'V [km/s]', size=fs)
    plt.text(textx[1], texty, r'$\sigma$ [km/s]', size=fs)

    plt.text(textx[0], texty2, str(int(sols[0])), size=fs)
    plt.text(textx[1], texty2, str(int(sols[1])), size=fs)

    fig.set_size_inches(18.5, 10.5)
    plt.savefig(save_path+spec_name+'_iteration_%d'%it,dpi=100)
    plt.close()
    
    return out
    
    



def bootstrap(nit, save_path, spec_name):
# basically bootstrap the noise!
    # Do one fit with flat noise, then save the best fit spectrum and the residuals.
    # Then, iterate ~200 times. For these iterations, set bias = 0.0. Each iteration, for each pixel, use the spectrum
    # value as the center of a gaussian and use the residuals at that pixel value as the width of the gaussian. Draw
    # from the resultant distribution to make the new noise. For each iteration, save the output V, sigma, h3, h4, and
    # print each spectrum so that we can see it evolving (it should look more like a real spectrum, rather than smooth
    # curve without lines)
    out = bootstrap_fig(save_path, spec_name, 0)
    sols = out[2]
    
    sol_matrix = np.zeros(shape=(2, nit))
    resid = out[1] - out[0]  # galaxy - bestfit = resid, from ppxf()
    #resid = resid[out[4]]    # resid[goodPixels]
    orig_noise = out[3]      # original noise of spectrum
    #orig_noise = orig_noise[out[4]] # original_noise[goodPixels]
    
    std_dev = np.sqrt(orig_noise**2 + resid**2)  # the residual noise and spectral noise added in quadrature
    bfit = out[0]
    orig = out[2]
    
    counter = 0
    
    for i in range(nit):
        #print(counter)
        new_gal = np.random.normal(loc=bfit, scale=std_dev)
        
        counter += 1
        new_gal = np.asarray(new_gal)
        #print(new_gal.shape)

        out = bootstrap_fig(save_path, spec_name, counter, galboots=new_gal)
        sols = out[2]
        sol_matrix[:, counter - 1] = sols  # save solutions
        plt.close("all")
        plt.clf()
        
    
    return sol_matrix, orig

def plot_boots(nit, sol_matrix, orig_fit,
                  save_path):
    iters=nit
    fig = plt.figure(figsize=(1.4, 3.8))
    ax1 = plt.subplot(1, 2, 1)
    ax2 = plt.subplot(1, 2, 2)

    axes = [ax1, ax2]
    labels = [r'V', r'$\sigma$',]
    for i in range(len(sol_matrix)):  # 4
        percs = np.percentile(sol_matrix[i], [0.3, 2.4, 16.0, 50.0, 84.0, 97.6, 99.7])  # median, 1,2,3 sigma
        t_x = min(sol_matrix[i])
        t_y1 = 0.02
        t_y2 = 0.03
        t_y3 = 0.04
        # axes[i].text(t_x, t_y3,)

        axes[i].axvline(x=orig_fit[i], color='k', linestyle='--', lw=2, label='Original fit')

        axes[i].hist(sol_matrix[i], bins=50, histtype="step", weights=[1. / iters] * len(sol_matrix[i]),
                     color='k', lw=2, label=labels[i])
        axes[i].axvline(x=percs[3], color='b', linestyle='--', lw=2, label='Median')
        axes[i].axvline(x=np.mean(sol_matrix[i]), color='b', ls='-', lw=2, label='Mean')

        axes[i].axvspan(percs[0], percs[6], color='b', alpha=0.1)  # 3 sigma
        axes[i].axvspan(percs[1], percs[5], color='b', alpha=0.1)  # 2 sigma
        axes[i].axvspan(percs[2], percs[4], color='b', alpha=0.1)  # 1 sigma
        axes[i].legend(numpoints=1, loc='upper left', prop={'size': 10})

        fig.set_size_inches(18.5, 6.5)
        # plt.text()

        #print(percs)
        #print(orig, 'orig')
    axes[0].set_xlabel(r'V [km/s]')
    axes[1].set_xlabel(r'$\sigma$ [km/s]')
    plt.savefig (save_path+'Resume_%d_iterations'%iters)  # , dpi=500)
    #plt.show()
    plt.close("all")
    plt.clf()
    

In [6]:
nit = 200
for i in range(len(save_paths)):
    spec_name = ("Bin"+save_paths[i][len(save_paths[i])-2])
    sol_matrix, orig_fit = bootstrap(nit=nit, save_path=save_paths[i], spec_name=spec_name)
    plot_boots(nit=nit, sol_matrix=sol_matrix, orig_fit=orig_fit, save_path=save_paths[i])
    plt.clf()
    np.savetxt(save_paths[i]+"Sol_Matrix.txt", np.column_stack([sol_matrix[0],sol_matrix[1]]),
               header="Vel [km/s] \t Disp [km/s]",
               fmt="%.5e \t %.5e")
    gc.collect()

<Figure size 432x288 with 0 Axes>